# CrewAI Workshop

## 1. CrewAI和Seprper 搜索集成 (可选）
需要提供Serper API KEY
Step 1: init LLM

In [ ]:
from langchain_aws import ChatBedrock

def init_bedrock_chat(model_id='anthropic.claude-3-sonnet-20240229-v1:0', region_name='us-east-1'):

    model_kwargs =  { 
        "max_tokens": 4096,
        "temperature": 0.0
    }
    bedrock_chat = ChatBedrock(model_id=model_id, model_kwargs=model_kwargs, region_name=region_name)
    print("boto3 Bedrock client successfully created!")
    return bedrock_chat

bedrock_llm = init_bedrock_chat()
bedrock_llm.invoke('hello')

Step 2: Create tools

In [ ]:
import os
os.environ["SERPER_API_KEY"] = "XXXXXXX"

from crewai_tools import SerperDevTool
search_tool = SerperDevTool()

Step 3 Create Agent

In [ ]:
from crewai import Agent
# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_tool],
  llm=bedrock_llm
)
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  llm=bedrock_llm
)

Step 4 Create Task

In [ ]:
from crewai import Task

task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

Step 5 Kick Off

In [ ]:
from crewai import Crew
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

# 2. 使用CrewAI 构建 App Store Review
导入第三方库来获取对应的商店数据


In [ ]:
from google_play_scraper import search, Sort, reviews

In [ ]:
from langchain.agents import tool 
@tool
def get_google_app_id(app_name:str):
    """Tool to found Google App Store AppID by app name"""
    try:
        result = search(
            app_name,
            lang="en",  # defaults to 'en'
            country="us",  # defaults to 'us'
            n_hits=1  # defaults to 30 (= Google's maximum)
        )
        if len(result)>0:
            return result[0]['appId']
        else:
            return None

    except Exception as e:
        print(e)
        return None

In [ ]:
get_google_app_id('starrail')

In [ ]:

import json
from datetime import datetime
from langchain.agents import tool
from google_play_scraper import Sort, reviews


class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()
        return super().default(o)


def save_review(app_id, data):
    with open(f"output/{app_id}.json", "w", encoding="utf-8") as file:
        json.dump(data, file, cls=DateTimeEncoder,
                  ensure_ascii=False, indent=4)


@tool
def get_google_play_app_review(app_id: str, country: str = 'us', rank: int = -1, page: int = 1):
    """Tool to found Google App Store App reviews by app id ,country ,rank and page"""
    try:

        filter_score = None
        if rank > 0 and rank < 6:
            filter_score = rank
        print(f"{country=}")
        result, continuation_token = reviews(
            app_id,
            lang='en',  # defaults to 'en'
            country="us" if country == "" else country,  # defaults to 'us'
            sort=Sort.NEWEST,  # defaults to Sort.NEWEST
            count=100,  # defaults to 100
            filter_score_with=filter_score
        )

        if continuation_token:
            print(f'Have continuation token: {continuation_token}')

        save_review(app_id, result)

        return [{"username": review["userName"], "content": review["content"], "score": review["score"]} for review in result]
    except Exception as e:
        print(e)
        return []

In [ ]:
get_google_play_app_review('com.miHoYo.GenshinImpact')

In [ ]:
from crewai import Agent
review_loader = Agent(
    role='App Review Research Analyst',
    goal='If user input is app id, you load App review  from a app id',
    backstory='Specializes in handling and interpreting app reviews documents',
    verbose=True,
    tools=[get_google_play_app_review],
    allow_delegation=False,
    llm=bedrock_llm,
    memory=True,
)

researcher = Agent(
  role='Expert App Store reviews summarizer',
  goal='Found prices issue , network issue, game content issue in every user\'s reviews content, you need keep username',
  backstory="""You are a renowned expert summarizer of AI related 
  Google App Store User reviews, known for your insightful 
  and engaging articles. You transform complex concepts into 
  compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  llm=bedrock_llm
  
)

In [ ]:
import sys
from crewai import Task, Crew

def init_app_crew(store:str,app_name:str,country:str, rank:int=-1,page:int=1, file:str="",output_stream=None):
    """
    This function is the main entry point for the application.
    It takes an app name and a review rank , page as input, and returns the result of processing the app reviews.
    """
    
    if output_stream is not None:
            sys.stdout = output_stream
    

    # Get the app ID from the app name using the get_google_app_id tool
    app_id = get_google_app_id(app_name)
        
    # Create a task for the review_reader agent to fetch reviews for the given app ID and rank
    review_loading_task = Task(
        description=f"""Use store: {store} ,app_name:{app_name} app_id: {app_id} and country:{country}, rank: {rank}, page: {page}""",
        agent=review_loader,
        expected_output='A refined finalized version of the blog post in markdown format'
    )

    content_creation_task = Task(
        description="""Using the insights provided by the research Analyst agent, if have different score , please calculate every score the percentage
        develop an engaging  content, you need use markdown format and translate to chinese. """,
        agent=researcher,
        expected_output=f'A refined finalized version of the blog post in markdown format,need include store: {store} , app_id: {app_id} and country:{country},translate to chinese.'
    )

    # Instantiate a Crew with the review_reader and researcher agents, and the tasks to be performed
    # Set the logging level to 2 (more verbose)
    crew = Crew(
        agents=[review_loader, researcher],
        tasks=[review_loading_task, content_creation_task],
        verbose=2, 
    )

    return app_id,crew

In [ ]:
app_id, crew=init_app_crew('Google Play', 'genshin', "us", -1)

In [ ]:
result = crew.kickoff()

将代码整合到一起

In [ ]:
from langchain_aws import ChatBedrock
from langchain.agents import tool 
from google_play_scraper import search, Sort, reviews
import json
from datetime import datetime
from langchain.agents import tool
from google_play_scraper import Sort, reviews
from crewai import Agent
import sys
from crewai import Task, Crew

def init_bedrock_chat(model_id='anthropic.claude-3-sonnet-20240229-v1:0', region_name='us-east-1'):

    model_kwargs =  { 
        "max_tokens": 4096,
        "temperature": 0.0
    }
    bedrock_chat = ChatBedrock(model_id=model_id, model_kwargs=model_kwargs, region_name=region_name)
    print("boto3 Bedrock client successfully created!")
    return bedrock_chat


bedrock_llm = init_bedrock_chat()


@tool
def get_google_app_id(app_name:str):
    """Tool to found Google App Store AppID by app name"""
    try:
        result = search(
            app_name,
            lang="en",  # defaults to 'en'
            country="us",  # defaults to 'us'
            n_hits=1  # defaults to 30 (= Google's maximum)
        )
        if len(result)>0:
            return result[0]['appId']
        else:
            return None

    except Exception as e:
        print(e)
        return None



class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()
        return super().default(o)


def save_review(app_id, data):
    with open(f"output/{app_id}.json", "w", encoding="utf-8") as file:
        json.dump(data, file, cls=DateTimeEncoder,
                  ensure_ascii=False, indent=4)


@tool
def get_google_play_app_review(app_id: str, country: str = 'us', rank: int = -1, page: int = 1):
    """Tool to found Google App Store App reviews by app id ,country ,rank and page"""
    try:

        filter_score = None
        if rank > 0 and rank < 6:
            filter_score = rank
        print(f"{country=}")
        result, continuation_token = reviews(
            app_id,
            lang='en',  # defaults to 'en'
            country="us" if country == "" else country,  # defaults to 'us'
            sort=Sort.NEWEST,  # defaults to Sort.NEWEST
            count=100,  # defaults to 100
            filter_score_with=filter_score
        )

        if continuation_token:
            print(f'Have continuation token: {continuation_token}')

        # if len(result) > 0:
        #     process_app_review_with_llm(result)
        save_review(app_id, result)
        # with open(f"output/{app_id}.json", "w", encoding="utf-8") as file:
        #     json.dump(result, file, cls=DateTimeEncoder,ensure_ascii=False, indent=4)

        return [{"username": review["userName"], "content": review["content"], "score": review["score"]} for review in result]
    except Exception as e:
        print(e)
        return []
    

review_loader = Agent(
    role='App Review Research Analyst',
    goal='If user input is app id, you load App review  from a app id',
    backstory='Specializes in handling and interpreting app reviews documents',
    verbose=True,
    tools=[get_google_play_app_review],
    allow_delegation=False,
    llm=bedrock_llm,
    memory=True,
)

researcher = Agent(
  role='Expert App Store reviews summarizer',
  goal='Found prices issue , network issue, game content issue in every user\'s reviews content, you need keep username',
  backstory="""You are a renowned expert summarizer of AI related 
  Google App Store User reviews, known for your insightful 
  and engaging articles. You transform complex concepts into 
  compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  llm=bedrock_llm
  
)



def init_app_crew(store:str,app_name:str,country:str, rank:int=-1,page:int=1, file:str="",output_stream=None):
    """
    This function is the main entry point for the application.
    It takes an app name and a review rank , page as input, and returns the result of processing the app reviews.
    """
    
    if output_stream is not None:
            sys.stdout = output_stream
    

    # Get the app ID from the app name using the get_google_app_id tool
    app_id = get_google_app_id(app_name)
        
    # Create a task for the review_reader agent to fetch reviews for the given app ID and rank
    review_loading_task = Task(
        description=f"""Use store: {store} ,app_name:{app_name} app_id: {app_id} and country:{country}, rank: {rank}, page: {page}""",
        agent=review_loader,
        expected_output='A refined finalized version of the blog post in markdown format'
    )

    content_creation_task = Task(
        description="""Using the insights provided by the research Analyst agent, if have different score , please calculate every score the percentage
        develop an engaging  content, you need use markdown format and translate to chinese. """,
        agent=researcher,
        expected_output=f'A refined finalized version of the blog post in markdown format,need include store: {store} , app_id: {app_id} and country:{country},translate to chinese.'
    )

    # Instantiate a Crew with the review_reader and researcher agents, and the tasks to be performed
    # Set the logging level to 2 (more verbose)
    crew = Crew(
        agents=[review_loader, researcher],
        tasks=[review_loading_task, content_creation_task],
        verbose=2, 
    )

    return app_id,crew

app_id, crew=init_app_crew('Google Play', 'genshin', "us", -1)
crew.kickoff()

# 3 启动GUI
使用streamlit 启动GUI, 需要创建一个output目录

In [ ]:
!mkdir output 